In [1]:
!pip install cvxopt -q
!pip install optuna -q

In [2]:
import sklearn
from sklearn.preprocessing import OneHotEncoder
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt

import random
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import scale
import optuna
from sklearn.metrics import accuracy_score
from sklearn.metrics import recall_score, precision_score
from numpy import linalg
import cvxopt
import cvxopt.solvers
import sklearn

import os
cvxopt.solvers.options['show_progress'] = False

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [3]:
xtr0=pd.read_csv("/kaggle/input/kernel-methods-ammi-2023/Xtr0.csv", sep=',', index_col=0)
xtr1=pd.read_csv("/kaggle/input/kernel-methods-ammi-2023/Xtr1.csv",sep=',', index_col=0)
xtr2=pd.read_csv("/kaggle/input/kernel-methods-ammi-2023/Xtr2.csv", sep=',', index_col=0)
Xte0=pd.read_csv("/kaggle/input/kernel-methods-ammi-2023/Xte0.csv",sep=',', index_col=0)
Xte1=pd.read_csv("/kaggle/input/kernel-methods-ammi-2023/Xte1.csv",sep=',', index_col=0)
Xte2=pd.read_csv("/kaggle/input/kernel-methods-ammi-2023/Xte2.csv",sep=',', index_col=0)
Ytr0=pd.read_csv("//kaggle/input/kernel-methods-ammi-2023/Ytr0.csv",sep=',', index_col=0)
Ytr1=pd.read_csv("/kaggle/input/kernel-methods-ammi-2023/Ytr0.csv",sep=',', index_col=0)
Ytr2=pd.read_csv("/kaggle/input/kernel-methods-ammi-2023/Ytr0.csv",sep=',', index_col=0)

In [4]:
print("xtr0 shape is ", xtr0.shape)
print("xtr1 shape is ", xtr1.shape)
print("xtr0 shape is ", xtr2.shape)
print("Xte0 shape is ", Xte0.shape)
print("Xte1 shape is ", Xte1.shape)
print("Xte2 shape is ", Xte2.shape)
print("Ytr0 shape is", Ytr0.shape)
print("Ytr1 shape is", Ytr1.shape)
print("Ytr2 shape is", Ytr2.shape)

xtr0 shape is  (2000, 1)
xtr1 shape is  (2000, 1)
xtr0 shape is  (2000, 1)
Xte0 shape is  (1000, 1)
Xte1 shape is  (1000, 1)
Xte2 shape is  (1000, 1)
Ytr0 shape is (2000, 1)
Ytr1 shape is (2000, 1)
Ytr2 shape is (2000, 1)


In [5]:
Xte0.shape

(1000, 1)

In [6]:
X_train_=np.vstack((xtr0,xtr1,xtr2,))
y=np.vstack((Ytr0,Ytr1,Ytr2))

In [7]:
X_train_.shape

(6000, 1)

In [8]:
y.shape

(6000, 1)

In [9]:
# y['Bound'] = y.Bound.apply(lambda x: -1 if x == 0 else 1)
# y.head()
# y = y.Bound.values
# y

In [10]:
# Step 1
X_train_ = np.vstack((xtr0, xtr1, xtr2))
y = np.vstack((Ytr0, Ytr1, Ytr2))

# Step 2
X_train_.shape  # (5000, 1)
y.shape  # (6000, 1)

# Transform y
y = np.where(y == 0, -1, 1)

# Check the first few elements of y
print(y[:5])

# Continue with your SVM implementation...


[[-1]
 [ 1]
 [ 1]
 [ 1]
 [ 1]]


In [11]:
def getKmers(sequence, kmer_size):
    kmers = []
    n = len(sequence)
    for i in range(n - kmer_size + 1):
        kmer = sequence[i:i + kmer_size]
        kmers.append(kmer)
    return kmers


def base2int(c):
    return {'a':0,'c':1,'g':2,'t':3}.get(c,0)

def index(kmer):
    base_idx = np.array([base2int(base) for base in kmer])
    multiplier = 4** np.arange(len(kmer))
    kmer_idx = multiplier.dot(base_idx)
    return kmer_idx
# The index_kmer function with a corrected name to avoid conflict
def index_kmer(kmer):
    base_idx = np.array([base2int(base) for base in kmer])
    multiplier = 4 ** np.arange(len(kmer))
    kmer_idx = multiplier.dot(base_idx)
    return kmer_idx


def spectral_embedding(sequence, kmer_size=3):
    kmers = getKmers(sequence, kmer_size=kmer_size)
    kmer_idxs = [index(kmer) for kmer in kmers]
    one_hot_vector = np.zeros(4**kmer_size)
    one_hot_vector[kmer_idxs] = 1
    return one_hot_vector


In [12]:
def rbf_kernel_element_wise(x, y, sigma=1):
    K =  np.exp(-np.sum((x-y)**2)/(2*sigma**2))
    return K

def rbf_kernel(X1, X2, sigma=10):
    X2_norm = np.sum(X2 ** 2, axis = -1)
    X1_norm = np.sum(X1 ** 2, axis = -1)
    gamma = 1 / (2 * sigma ** 2)
    K = np.exp(- gamma * (X1_norm[:, None] + X2_norm[None, :] - 2 * np.dot(X1, X2.T)))
    return K

def sigma_from_median(X):
    pairwise_diff = X[:, :, None] - X[:, :, None].T
    pairwise_diff *= pairwise_diff
    euclidean_dist = np.sqrt(pairwise_diff.sum(axis=1))
    return np.median(euclidean_dist)

def gaussian_kernel(x, y, sigma=5.0):
    return np.exp(-np.linalg.norm(x-y)**2 / (2 * (sigma ** 2)))

def linear_kernel(x1, x2):
    return np.dot(x1, x2)

def polynomial_kernel(X1, X2, power=2):
    return np.power((1 + linear_kernel(X1, X2)),power)


def rbf_kernel(X1, X2, sigma=10):
    X2_norm = np.sum(X2 ** 2, axis = -1)
    X1_norm = np.sum(X1 ** 2, axis = -1)
    gamma = 1 / (2 * sigma ** 2)
    K = np.exp(- gamma * (X1_norm[:, None] + X2_norm[None, :] - 2 * np.dot(X1, X2.T)))
    return K

def sigma_from_median(X):
    pairwise_diff = X[:, :, None] - X[:, :, None].T
    pairwise_diff *= pairwise_diff
    euclidean_dist = np.sqrt(pairwise_diff.sum(axis=1))
    return np.median(euclidean_dist)

def linear_kernel(X1, X2):
    return X1.dot(X2.T)

In [13]:
class KernelMethodBase(object):
    '''
    Base class for kernel methods models

    Methods
    ----
    fit
    predict
    '''
    kernels_ = {
        'linear': linear_kernel,
        'polynomial': polynomial_kernel,
        'rbf': rbf_kernel,
        'gaussian':gaussian_kernel
    }
    def __init__(self, kernel='linear', **kwargs):
        self.kernel_name = kernel
        self.kernel_function_ = self.kernels_[kernel]
        self.kernel_parameters = self.get_kernel_parameters(**kwargs)

    def get_kernel_parameters(self, **kwargs):
        params = {}
        if self.kernel_name == 'rbf' or self.kernel_name == 'gaussian':
            params['sigma'] = kwargs.get('sigma', None)
        if self.kernel_name == 'polynomial':
            params['power'] = kwargs.get('power', None)


        return params

    def fit(self, X, y, **kwargs):
        return self

    def decision_function(self, X):
        pass

    def predict(self, X):
        pass

In [14]:
import cvxopt
import numpy as np

class KernelSVM(KernelMethodBase):
    def __init__(self, C=0.1, **kwargs):
        self.C = C
        # Python 3: replace the following line by
        # super().__init__(**kwargs)
        super(KernelSVM, self).__init__(**kwargs)

    def cvxopt_qp(self, P, q, G, h, A, b):
        P = .5 * (P + P.T)
        cvx_matrices = [
            cvxopt.matrix(M) if M is not None else None for M in [P, q, G, h, A, b]
        ]
        #cvxopt.solvers.options['show_progress'] = False
        solution = cvxopt.solvers.qp(*cvx_matrices, options={'show_progress': False})
        return np.array(solution['x']).flatten()

    def svm_dual_soft_to_qp_kernel(self, K, y, C=1):
        n = K.shape[0]
        assert (len(y) == n)

        # Dual formulation, soft margin
        # P = np.diag(y) @ K @ np.diag(y)
        P = np.diag(y).dot(K).dot(np.diag(y))
        # As a regularization, we add epsilon * identity to P
        eps = 1e-12
        P += eps * np.eye(n)
        q = - np.ones(n)
        G = np.vstack([-np.eye(n), np.eye(n)])
        h = np.hstack([np.zeros(n), C * np.ones(n)])
        A = y[np.newaxis, :]
        b = np.array([0.])
        return P, q, G, h, A.astype(float), b

    def fit(self, X, y, tol=1e-8):
        n = len(X)
        assert (n == len(y))

        self.X_train = X
        self.y_train = y

        # Kernel matrix
        K = self.kernel_function_(
            self.X_train, self.X_train, **self.kernel_parameters)

        # Solve dual problem
        self.alpha = self.cvxopt_qp(*self.svm_dual_soft_to_qp_kernel(K, y, C=self.C))

        # Compute support vectors and bias b
        sv = np.logical_and((self.alpha > tol), (self.C - self.alpha > tol))
        self.bias = y[sv] - K[sv].dot(self.alpha * y)
        self.bias = self.bias.mean()

        self.support_vector_indices = np.nonzero(sv)[0]

        return self

    def decision_function(self, X):
        K_x = self.kernel_function_(X, self.X_train, **self.kernel_parameters)
        return K_x.dot(self.alpha * self.y_train) + self.bias

    def predict(self, X):
        return np.sign(self.decision_function(X))


In [15]:
def cross_validate(x_data,y_data,lr=None,kernel=None,lambd=0.2,C=3,sigma=0.5,k=5,power=2):
    if len(x_data)%k != 0:
        print('cant vsplit',len(x_data),' by ',k)
        return

    x_data_splitted = np.vsplit(x_data,k)
    y_data_splitted = np.vsplit(y_data.reshape(-1,1),k)

    aggrigate_result = []
    for i in range(len(x_data_splitted)):
        train = []
        test = []
        items = [j for j in range(len(x_data_splitted)) if j !=i ]
        x_test = x_data_splitted[i]
        y_test = y_data_splitted[i]
        for item in items:
            if len(train) == 0:
                x_train = x_data_splitted[item]
                y_train = y_data_splitted[item]
            else:
                x_train = np.concatenate((x_train,x_data_splitted[item]), axis=0)
                y_train = np.concatenate((y_train,y_data_splitted[item]), axis=0)

        model = KernelSVM(C=C,
                          kernel=kernel,
                          lambd=lambd,
                          sigma=sigma,
                          power=power)
        model.fit(x_train, y_train.flatten())
        y_pred = model.predict(x_test)

        result = sum((y_pred.flatten()==y_test.flatten()))/len(y_test)

        aggrigate_result.append(result)

        value = sum(aggrigate_result)/len(aggrigate_result)
    return value

In [16]:
def get_data(X, kmer_size):
    kmer_data = np.zeros((len(X), 4**kmer_size))  # Initialize an array to store the kmer data for each sample
    for i, sequence in enumerate(X):
        embedding = spectral_embedding(sequence, kmer_size=kmer_size)
        if embedding is None or embedding.shape != (4**kmer_size,):
            # Handle cases where the shape of the embedding is not as expected
            # You may need to reshape, truncate, or pad the embedding accordingly
            raise ValueError(f"Invalid shape of embedding for sequence {i}")
        kmer_data[i] = embedding
    return kmer_data



# def get_train_test(X, y, p):
#     X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=p, random_state=42)
#     print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)
#     return X_train, X_test, y_train, y_test

def get_train_test(X, y, p):
    # Ensure that the number of samples in X and y are the same
    assert len(X) == len(y)

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=p, random_state=42)
    print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)
    return X_train, X_test, y_train, y_test


xtr0=pd.read_csv("/kaggle/input/kernel-methods-ammi-2023/Xtr0.csv", sep=',', index_col=0)
xtr1=pd.read_csv("/kaggle/input/kernel-methods-ammi-2023/Xtr1.csv",sep=',', index_col=0)
xtr2=pd.read_csv("/kaggle/input/kernel-methods-ammi-2023/Xtr2.csv", sep=',', index_col=0)
Xte0=pd.read_csv("/kaggle/input/kernel-methods-ammi-2023/Xte0.csv",sep=',', index_col=0)
Xte1=pd.read_csv("/kaggle/input/kernel-methods-ammi-2023/Xte1.csv",sep=',', index_col=0)
Xte2=pd.read_csv("/kaggle/input/kernel-methods-ammi-2023/Xte2.csv",sep=',', index_col=0)
Ytr0=pd.read_csv("//kaggle/input/kernel-methods-ammi-2023/Ytr0.csv",sep=',', index_col=0)
Ytr1=pd.read_csv("/kaggle/input/kernel-methods-ammi-2023/Ytr0.csv",sep=',', index_col=0)
Ytr2=pd.read_csv("/kaggle/input/kernel-methods-ammi-2023/Ytr0.csv",sep=',', index_col=0)
Xtr3 = pd.read_csv("/kaggle/input/kernel-methods-ammi-2023/Xtr0_mat100.csv", sep=",", index_col=0)
Xtr4 = pd.read_csv("/kaggle/input/kernel-methods-ammi-2023/Xtr1_mat100.csv", sep=",", index_col=0)
Xtr5 = pd.read_csv("/kaggle/input/kernel-methods-ammi-2023/Xtr2_mat100.csv", sep=",", index_col=0)
Xte3 = pd.read_csv("/kaggle/input/kernel-methods-ammi-2023/Xte0_mat100.csv", sep=",", index_col=0)
Xte4 = pd.read_csv("/kaggle/input/kernel-methods-ammi-2023/Xte1_mat100.csv", sep=",", index_col=0)
Xte5 = pd.read_csv("/kaggle/input/kernel-methods-ammi-2023/Xte2_mat100.csv", sep=",", index_col=0)

# Use 2000 samples from xtr0 and Ytr0
X_train0, Xte0, y_train0, y_test0 = get_train_test(get_data(xtr0[:2000], 6), Ytr0.values[:2000], p=0.3)

# Use 2000 samples from xtr1 and Ytr1
X_train1, Xte1, y_train1, y_test1 = get_train_test(get_data(xtr1[:2000], 6), Ytr1.values[:2000], p=0.3)

# Use 2000 samples from xtr2 and Ytr2
X_train2, Xte2, y_train2, y_test2 = get_train_test(get_data(xtr2[:2000], 6), Ytr2.values[:2000], p=0.3)

# Use 2000 samples from Xtr3 and Ytr0
X_train3, Xte3, y_train3, y_test3 = get_train_test(get_data(Xtr3[:1400], 6), Ytr0.values[:1400], p=0.3)

# Use 2000 samples from Xtr4 and Ytr1
X_train4, Xte4, y_train4, y_test4 = get_train_test(get_data(Xtr4[:1400], 6), Ytr1.values[:1400], p=0.3)

# Use 2000 samples from Xtr5 and Ytr2
X_train5, Xte5, y_train5, y_test5 = get_train_test(get_data(Xtr5[:1400], 6), Ytr2.values[:1400], p=0.3)

print(X_train0.shape, Xte0.shape, y_train0.shape, y_test0.shape)
print(X_train1.shape, Xte1.shape, y_train1.shape, y_test1.shape)
print(X_train2.shape, Xte2.shape, y_train2.shape, y_test2.shape)
print(X_train3.shape, Xte3.shape, y_train3.shape, y_test3.shape)
print(X_train4.shape, Xte4.shape, y_train4.shape, y_test4.shape)
print(X_train5.shape, Xte5.shape, y_train5.shape, y_test5.shape)


(1400, 4096) (600, 4096) (1400, 1) (600, 1)
(1400, 4096) (600, 4096) (1400, 1) (600, 1)
(1400, 4096) (600, 4096) (1400, 1) (600, 1)
(980, 4096) (420, 4096) (980, 1) (420, 1)
(980, 4096) (420, 4096) (980, 1) (420, 1)
(980, 4096) (420, 4096) (980, 1) (420, 1)
(1400, 4096) (600, 4096) (1400, 1) (600, 1)
(1400, 4096) (600, 4096) (1400, 1) (600, 1)
(1400, 4096) (600, 4096) (1400, 1) (600, 1)
(980, 4096) (420, 4096) (980, 1) (420, 1)
(980, 4096) (420, 4096) (980, 1) (420, 1)
(980, 4096) (420, 4096) (980, 1) (420, 1)


In [17]:

xtr0=pd.read_csv("/kaggle/input/kernel-methods-ammi-2023/Xtr0.csv", sep=',', index_col=0)
xtr1=pd.read_csv("/kaggle/input/kernel-methods-ammi-2023/Xtr1.csv",sep=',', index_col=0)
xtr2=pd.read_csv("/kaggle/input/kernel-methods-ammi-2023/Xtr2.csv", sep=',', index_col=0)
Xte0=pd.read_csv("/kaggle/input/kernel-methods-ammi-2023/Xte0.csv",sep=',', index_col=0)
Xte1=pd.read_csv("/kaggle/input/kernel-methods-ammi-2023/Xte1.csv",sep=',', index_col=0)
Xte2=pd.read_csv("/kaggle/input/kernel-methods-ammi-2023/Xte2.csv",sep=',', index_col=0)
Ytr0=pd.read_csv("//kaggle/input/kernel-methods-ammi-2023/Ytr0.csv",sep=',', index_col=0)
Ytr1=pd.read_csv("/kaggle/input/kernel-methods-ammi-2023/Ytr0.csv",sep=',', index_col=0)
Ytr2=pd.read_csv("/kaggle/input/kernel-methods-ammi-2023/Ytr0.csv",sep=',', index_col=0)
Xtr3 = pd.read_csv("/kaggle/input/kernel-methods-ammi-2023/Xtr0_mat100.csv", sep=",", index_col=0)
Xtr4 = pd.read_csv("/kaggle/input/kernel-methods-ammi-2023/Xtr1_mat100.csv", sep=",", index_col=0)
Xtr5 = pd.read_csv("/kaggle/input/kernel-methods-ammi-2023/Xtr2_mat100.csv", sep=",", index_col=0)
Xte3 = pd.read_csv("/kaggle/input/kernel-methods-ammi-2023/Xte0_mat100.csv", sep=",", index_col=0)
Xte4 = pd.read_csv("/kaggle/input/kernel-methods-ammi-2023/Xte1_mat100.csv", sep=",", index_col=0)
Xte5 = pd.read_csv("/kaggle/input/kernel-methods-ammi-2023/Xte2_mat100.csv", sep=",", index_col=0)

# Use 2000 samples from xtr0 and Ytr0
X_train0, Xte0, y_train0, y_test0 = get_train_test(get_data(xtr0[:2000], 6), Ytr0.values[:2000], p=0.3)

# Use 2000 samples from xtr1 and Ytr1
X_train1, Xte1, y_train1, y_test1 = get_train_test(get_data(xtr1[:2000], 6), Ytr1.values[:2000], p=0.3)

# Use 2000 samples from xtr2 and Ytr2
X_train2, Xte2, y_train2, y_test2 = get_train_test(get_data(xtr2[:2000], 6), Ytr2.values[:2000], p=0.3)

# Use 2000 samples from Xtr3 and Ytr0
X_train3, Xte3, y_train3, y_test3 = get_train_test(get_data(Xtr3[:1400], 6), Ytr0.values[:1400], p=0.3)

# Use 2000 samples from Xtr4 and Ytr1
X_train4, Xte4, y_train4, y_test4 = get_train_test(get_data(Xtr4[:1400], 6), Ytr1.values[:1400], p=0.3)

# Use 2000 samples from Xtr5 and Ytr2
X_train5, Xte5, y_train5, y_test5 = get_train_test(get_data(Xtr5[:1400], 6), Ytr2.values[:1400], p=0.3)

print(X_train0.shape, Xte0.shape, y_train0.shape, y_test0.shape)
print(X_train1.shape, Xte1.shape, y_train1.shape, y_test1.shape)
print(X_train2.shape, Xte2.shape, y_train2.shape, y_test2.shape)
print(X_train3.shape, Xte3.shape, y_train3.shape, y_test3.shape)
print(X_train4.shape, Xte4.shape, y_train4.shape, y_test4.shape)
print(X_train5.shape, Xte5.shape, y_train5.shape, y_test5.shape)


(1400, 4096) (600, 4096) (1400, 1) (600, 1)
(1400, 4096) (600, 4096) (1400, 1) (600, 1)
(1400, 4096) (600, 4096) (1400, 1) (600, 1)
(980, 4096) (420, 4096) (980, 1) (420, 1)
(980, 4096) (420, 4096) (980, 1) (420, 1)
(980, 4096) (420, 4096) (980, 1) (420, 1)
(1400, 4096) (600, 4096) (1400, 1) (600, 1)
(1400, 4096) (600, 4096) (1400, 1) (600, 1)
(1400, 4096) (600, 4096) (1400, 1) (600, 1)
(980, 4096) (420, 4096) (980, 1) (420, 1)
(980, 4096) (420, 4096) (980, 1) (420, 1)
(980, 4096) (420, 4096) (980, 1) (420, 1)


In [18]:
import numpy as np

def objective(trial):
    # Hyperparameter search space
    sigma = trial.suggest_float('sigma', 1, 100)
    power = trial.suggest_float('power', 1, 10)
    C = trial.suggest_float('C', 1, 100)
    kmer_size = trial.suggest_int('kmer_size', 1, 10)

    # Initialize lists to store kmer data and labels for each dataset
    kmer_data_train = []
    y_train_list = []

    # Loop over the datasets
    for xtr, ytr in [(xtr0, Ytr0), (xtr1, Ytr1), (xtr2, Ytr2)]:
        # Call get_data with the determined kmer_size for the current dataset
        kmer_data = get_data(xtr[:5000], kmer_size)
        y_train = ytr.values[:5000]

        # Append the kmer data and labels to the lists
        kmer_data_train.append(kmer_data)
        y_train_list.append(y_train)

    # Concatenate the kmer data and labels for all datasets
    X_train = np.concatenate(kmer_data_train, axis=0)
    y_train = np.concatenate(y_train_list, axis=0)

    print(X_train.shape)  # Should be (15000, kmer_size)
    print(y_train.shape)  # Should be (15000, 1)

    # Return the result of cross_validate
    return cross_validate(X_train, y_train, kernel='polynomial', k=4, sigma=sigma, power=power, C=C)


In [ ]:

# Create an Optuna study and optimize the objective function
sampler = optuna.samplers.TPESampler()
study = optuna.create_study(sampler=sampler, direction='maximize')
df = study.optimize(func=objective, n_trials=5, show_progress_bar=True)


[I 2023-07-20 16:05:55,730] A new study created in memory with name: no-name-246b188d-9ca3-410f-9061-debc87345593


  0%|          | 0/5 [00:00<?, ?it/s]

(6000, 4096)
(6000, 1)
[I 2023-07-20 16:06:11,458] Trial 0 finished with value: 0.11833333333333333 and parameters: {'sigma': 61.51637539973272, 'power': 7.037651723573089, 'C': 67.42366798550819, 'kmer_size': 6}. Best is trial 0 with value: 0.11833333333333333.


In [ ]:


# Concatenate the training datasets and labels
X_train = np.concatenate([X_train0, X_train1, X_train2, X_train3,X_train4,X_train5])
y_train = np.concatenate([y_train0, y_train1, y_train2])

In [ ]:
import numpy as np

# Sample DNA sequences as strings
sequence_1 = "ATCGATCGATCG"
sequence_2 = "CGATCGATCGAT"
sequence_3 = "GATCGATCGATC"

# Create X_train as a list containing the DNA sequences
X_train = [sequence_1, sequence_2, sequence_3]

# Define a mapping for one-hot encoding
base2int = {'A': 0, 'T': 1, 'C': 2, 'G': 3}

# Function to convert a DNA sequence to a one-hot encoded vector
def one_hot_encode(sequence):
    sequence_length = len(sequence)
    encoding = np.zeros((sequence_length, 4), dtype=int)
    for i, base in enumerate(sequence):
        encoding[i, base2int[base]] = 1
    return encoding

# Encode each DNA sequence in X_train
X_encoded = [one_hot_encode(sequence) for sequence in X_train]

# Convert the list of encoded sequences to a numpy array
X_train_encoded = np.array(X_encoded)

print(X_train_encoded.shape)  # Should output: (3, 12, 4)


In [ ]:
# Combine the datasets and labels
X_combined = np.concatenate([X_train0, X_train1, X_train2, X_train3,X_train4,X_train5])
y_combined = np.concatenate([y_train0, y_train1, y_train2]).ravel()

# Perform cross-validation on the combined data
cross_val_score = cross_validate(X_combined, y_combined, kernel='polynomial', k=4, sigma=50, power=2, C=1)

# Print the cross-validation score
print("Cross-validation score:", cross_val_score)


In [ ]:
from sklearn.metrics import accuracy_score

kernel = 'polynomial'
power = 2
sigma = 50.
C = 1.


In [ ]:
# Define the hyperparameters for the KernelSVM model
kernel = 'polynomial'
C = 1
sigma = 50
power = 2

# Create a KernelSVM model with the specified hyperparameters
model = KernelSVM(C=C, kernel=kernel, sigma=sigma, power=power)

# Concatenate the datasets and labels
X_combined = np.concatenate([X_train0, X_train1, X_train2])
y_combined = np.concatenate([y_train0, y_train1, y_train2]).ravel()

# Fit the model on the combined training data
model.fit(X_combined, y_combined)

# Make predictions on Xte0, Xte1, and Xte2
y_pred0 = model.predict(Xte0)
y_pred1 = model.predict(Xte1)
y_pred2 = model.predict(Xte2)
y_pred3 = model.predict(Xte3)
y_pred4 = model.predict(Xte4)
y_pred5 = model.predict(Xte5)

In [ ]:
print("y_pred0 shape:", y_pred0.shape)
print("y_pred1 shape:", y_pred1.shape)
print("y_pred2 shape:", y_pred2.shape)

# Take the first 3000 rows from each prediction for submission
y_pred0 = y_pred0[:3000].reshape(-1, 1)
y_pred1 = y_pred1[:3000].reshape(-1, 1)
y_pred2 = y_pred2[:3000].reshape(-1, 1)

print("y_pred0 shape after reshape:", y_pred0.shape)
print("y_pred1 shape after reshape:", y_pred1.shape)
print("y_pred2 shape after reshape:", y_pred2.shape)